In [1]:
!pip install opencv-python



In [5]:
!pip install pyttsx3


  Using cached pyttsx3-2.91-py3-none-any.whl.metadata (3.8 kB)
  Using cached pypiwin32-223-py3-none-any.whl.metadata (236 bytes)
Using cached pyttsx3-2.91-py3-none-any.whl (33 kB)
   ---------------------------------------- 0.0/226.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/226.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/226.8 kB ? eta -:--:--
   ----- --------------------------------- 30.7/226.8 kB 435.7 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/226.8 kB 871.5 kB/s eta 0:00:01
   ---------------------------------------  225.3/226.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 226.8/226.8 kB 1.3 MB/s eta 0:00:00
Using cached pypiwin32-223-py3-none-any.whl (1.7 kB)


In [7]:
!pip install requests


In [3]:
!pip install torch torchvision torchaudio  
!pip install opencv-python                 
!pip install pyyaml
!pip install yolov5                        


   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/199.4 MB 2.9 MB/s eta 0:01:09
   ---------------------------------------- 0.5/199.4 MB 5.2 MB/s eta 0:00:39
   ---------------------------------------- 0.9/199.4 MB 6.6 MB/s eta 0:00:31
   ---------------------------------------- 1.2/199.4 MB 6.7 MB/s eta 0:00:30
   ---------------------------------------- 1.9/199.4 MB 7.1 MB/s eta 0:00:28
   ---------------------------------------- 2.5/199.4 MB 7.8 MB/s eta 0:00:26
    --------------------------------------- 2.9/199.4 MB 8.1 MB/s eta 0:00:25
    --------------------------------------- 3.2/199.4 MB 8.2 MB/s eta 0:00:25
    --------------------------------------- 3.8/199.4 MB 8.7 MB/s eta 0:00:23
    --------------------------------------- 4.4/199.4 MB 8.7 MB/s eta 0:00:23
    --------------------------------------- 4.8/199.4 MB 9.0 MB/s eta 0:00:22


In [5]:
pip install yolov5

  Using cached yolov5-7.0.13-py37.py38.py39.py310-none-any.whl.metadata (10 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Using cached ultralytics-8.2.91-py3-none-any.whl.metadata (41 kB)
  Using cached fire-0.6.0.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached boto3-1.35.15-py3-none-any.whl.metadata (6.6 kB)
  Using cached sahi-0.11.18-py3-none-any.whl.metadata (17 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached roboflow-1.1.44-py3-none-any.whl.metadata (9.7 kB)
  Using cached botocore-1.35.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 k

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.35.15 which is incompatible.


In [ ]:
import cv2
import pyttsx3
import numpy as np

# Initialize TTS engine
engine = pyttsx3.init()

def speak(text):
    engine.say(text)
    engine.runAndWait()

def buzzer_sound(distance):
    # Adjust buzzer sound if needed; for now, it's omitted
    pass

# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Load class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Parameters for distance calculation
KNOWN_WIDTH = 0.5  # Example width of object in meters (adjust as needed)
FOCAL_LENGTH = 600  # Example focal length in pixels (adjust as needed)

def calculate_distance(width_in_frame):
    # Simple distance calculation based on the width of the object in the frame
    return (KNOWN_WIDTH * FOCAL_LENGTH) / width_in_frame

def detect_objects(frame):
    height, width = frame.shape[:2]
    # Lower the input resolution to (320, 320) for faster processing
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = int(scores.argmax())
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    detected_objects = []

    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            x, y, w, h = box
            label = str(classes[class_ids[i]])
            distance = calculate_distance(w)
            detected_objects.append((label, [x, y, w, h], distance))

    return detected_objects

def main():
    cap = cv2.VideoCapture(0)
    camera_on = True
    announced_objects = set()
    frame_skip = 2  # Skip every 2nd frame for faster processing
    frame_count = 0

    while True:
        if camera_on:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            if frame_count % frame_skip != 0:  # Skip frames to reduce load
                continue

            objects = detect_objects(frame)
            current_detected_objects = {(label, round(distance, 2)) for label, bbox, distance in objects}

            # Announce newly detected objects
            for label, bbox, distance in objects:
                object_id = (label, round(distance, 2))
                if object_id not in announced_objects:
                    x, y, w, h = bbox
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label} ({distance:.2f}m)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
                    speak(f"There is a {label} detected {distance:.2f} meters in front of you.")
                    announced_objects.add(object_id)  # Mark this object as announced

            # Update last detected objects
            announced_objects = {obj for obj in announced_objects if obj in current_detected_objects}

            cv2.imshow("YOLO Object Detection", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('t'):  # Toggle camera on/off
            camera_on = not camera_on
            if not camera_on:
                cap.release()
                cv2.destroyAllWindows()
            else:
                cap = cv2.VideoCapture(0)

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
